In [2]:
import collections, csv, math, os

Figs 3-16 and 18-31.
Input: LexiconHurvitzDataset.csv
Output: early_late_freqs_per_book.csv and relative_per_book.csv.

In [5]:
feature_list = []

late_books = ['ezek', 'hag', 'zech', 'mal', 'song', 'qoh', 'esth', 'dan', 'ezra', 'neh', 'chr', '1chr', '2chr']
core_late_books = ['1chr', '2chr', 'esth', 'dan', 'ezra', 'neh']
chron = ['1chr', '2chr']
synoptic = ['ChrS', 'ChrN']

The information we need is collected first in several dictionaries 

#early_freqs and late_freqs collect the frequency of early and late variants per book
early_freqs = collections.defaultdict(lambda:collections.Counter())
late_freqs = collections.defaultdict(lambda:collections.Counter())

#synop_early_freqs and synop_late_freqs collect the frequency of early and late variants in synoptic and non-synoptic Chronicles.
synop_late_freqs = collections.defaultdict(lambda:collections.Counter())
synop_early_freqs = collections.defaultdict(lambda:collections.Counter())

#relative_dict and relative_synop collect the proportion of each variable occurring in a book or in synoptic/non-synoptic Chronicles. 
relative_synop = collections.defaultdict(lambda:collections.Counter())
relative_dict = collections.defaultdict(lambda:collections.Counter())
core_late_books_relative_dict = collections.defaultdict(float)
core_late_books_late_dict = collections.defaultdict(int)
core_late_books_early_dict = collections.defaultdict(int)

with open('LexiconHurvitzDataset.csv', 'rt') as f:    
    reader = csv.reader(f)
    for row in reader:
        #first collect data for synoptic and non-synoptic Chronicles
        if row[3] == '1':
                if not row[1] == '7':
                    synop_late_freqs[row[8]][row[1]] += int(row[7])
        elif row[3] == '0':
            if not row[1] == '7':
                synop_early_freqs[row[8]][row[1]] += int(row[7])
        #now the rest, select late variants
        if row[3] == '1':
            if not row[1] in feature_list:
                feature_list.append(row[1])
            if not row[1] == '7':
                if row[4] in core_late_books:
                    core_late_books_late_dict[row[1]] += int(row[7])
                if row[4] in chron:
                   late_freqs['chr'][row[1]] += int(row[7]) 
                else:
                    late_freqs[row[4]][row[1]] += int(row[7])
        #select early variants
        elif row[3] == '0':
            if not row[1] == '7':
                if row[4] in core_late_books:
                    core_late_books_early_dict[row[1]] += int(row[7])
                if row[4] in chron:
                   early_freqs['chr'][row[1]] += int(row[7])
                else:    
                    early_freqs[row[4]][row[1]] += int(row[7])

#calculate the late fractions
for book in late_books:
    for feat in feature_list:
        if late_freqs[book][feat] + early_freqs[book][feat] > 0:
            relative_dict[book][feat] = late_freqs[book][feat]/(late_freqs[book][feat] + early_freqs[book][feat])

for syn in synoptic:
    for feat in feature_list:
        if synop_late_freqs[syn][feat] + synop_early_freqs[syn][feat] > 0:
            relative_synop[syn][feat] = synop_late_freqs[syn][feat]/ (synop_late_freqs[syn][feat] + synop_early_freqs[syn][feat])
            
for feat in feature_list:
    if (core_late_books_late_dict[feat] + core_late_books_early_dict[feat]) > 0:
        core_late_books_relative_dict[feat] = core_late_books_late_dict[feat]/ (core_late_books_late_dict[feat] + core_late_books_early_dict[feat])

#store all the information in a csv
csvh = open(r"early_late_freqs_per_book.csv", "w")
row = ['book', 'feature', 'early_late', 'frequency']
csvh.write('{}\n'.format(','.join(row)))
for book in late_books:
    for feat in feature_list:
        if not feat == '7':
            row = []
            row.append(book)
            row.append(feat)
            row.append('1')
            if feat in late_freqs[book].keys():
                row.append(str(late_freqs[book][feat]))
            else:
                row.append('0')
            csvh.write('{}\n'.format(','.join(row)))
            row = []
            row.append(book)
            row.append(feat)
            row.append('0')
            if feat in early_freqs[book].keys():
                row.append(str(-early_freqs[book][feat]))
            else:
                row.append('0')
            csvh.write('{}\n'.format(','.join(row))) 
for syn in synoptic:
    for feat in feature_list:
        if not feat == '7':
            row = []
            row.append(syn)
            row.append(feat)
            row.append('1')
            if feat in synop_late_freqs[syn].keys():
                row.append(str(synop_late_freqs[syn][feat]))
            else:
                row.append('0')
            csvh.write('{}\n'.format(','.join(row)))
            row = []
            row.append(syn)
            row.append(feat)
            row.append('0') 
            if feat in synop_early_freqs[syn].keys():
                row.append(str(-synop_early_freqs[syn][feat]))
            else:
                row.append('0')
            csvh.write('{}\n'.format(','.join(row)))
for feat in feature_list:
        if not feat == '7':
            row = []
            row.append('core_late_books')
            row.append(feat)
            row.append('1')
            if feat in core_late_books_late_dict.keys():
                row.append(str(core_late_books_late_dict[feat]))
            else:
                row.append('0')
            csvh.write('{}\n'.format(','.join(row)))
            row = []
            row.append('core_late_books')
            row.append(feat)
            row.append('0')
            if feat in core_late_books_early_dict.keys():
                row.append(str(-core_late_books_early_dict[feat]))
            else:
                row.append('0')
            csvh.write('{}\n'.format(','.join(row)))
            
csvh.close()

csvh = open(r"relative_per_book.csv", "w")
row = ['book', 'feature', 'frequency']
csvh.write('{}\n'.format(','.join(row)))
for book in late_books:
    for fe in list(relative_dict[book].keys()):
        if not fe == '7':
            row = []
            row.append(book)
            row.append(fe)
            row.append(str(relative_dict[book][fe]))
            csvh.write('{}\n'.format(','.join(row)))
for syn in synoptic:
    for fe in relative_synop[syn].keys():
        if not fe == '7':
            row = []
            row.append(syn)
            row.append(fe)
            row.append(str(relative_synop[syn][fe]))
            csvh.write('{}\n'.format(','.join(row)))
for fe in list(core_late_books_relative_dict.keys()):
        if not fe == '7':
            row = []
            row.append('core_late_books')
            row.append(fe)
            row.append(str(core_late_books_relative_dict[fe]))
            csvh.write('{}\n'.format(','.join(row)))
            
csvh.close()

For figure 17 we need the length of the books. We calculate these lengths using Laf Fabric. For more information about Laf Fabric, see shebanq.ancient-data.org.

In [6]:
import subprocess

from lxml import etree

import laf
from laf.fabric import LafFabric
from etcbc.preprocess import prepare
from etcbc.mql import MQL
fabric = LafFabric()

  0.00s This is LAF-Fabric 4.5.23
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: https://shebanq.ancient-data.org/static/docs/featuredoc/texts/welcome.html



In [7]:
API = fabric.load('etcbc4b', '--', 'mql', {
    "xmlids": {"node": False, "edge": False},
    "features": ('''
        oid otype monads number
        language lex g_word
        sp gn nu ps vt vs
        function typ det
        book chapter verse label
    ''',''),
    "prepare": prepare,
    "primary": False,
}, verbose='DEBUG')
exec(fabric.localnames.format(var='fabric'))
Q = MQL(API)

  0.00s LOADING API: please wait ... 
  0.05s DETAIL: COMPILING m: UP TO DATE
  0.13s USING main  DATA COMPILED AT: 2015-11-02T15-08-56
  0.13s DETAIL: COMPILING a: UP TO DATE
  0.21s DETAIL: load main: G.node_anchor_min
  0.31s DETAIL: load main: G.node_anchor_max
  0.40s DETAIL: load main: G.node_sort
  0.50s DETAIL: load main: G.node_sort_inv
  0.96s DETAIL: load main: G.edges_from
  1.08s DETAIL: load main: G.edges_to
  1.21s DETAIL: load main: F.etcbc4_db_monads [node] 
  1.96s DETAIL: load main: F.etcbc4_db_oid [node] 
  2.70s DETAIL: load main: F.etcbc4_db_otype [node] 
  3.37s DETAIL: load main: F.etcbc4_ft_det [node] 
  3.60s DETAIL: load main: F.etcbc4_ft_function [node] 
  3.72s DETAIL: load main: F.etcbc4_ft_g_word [node] 
  4.02s DETAIL: load main: F.etcbc4_ft_gn [node] 
  4.15s DETAIL: load main: F.etcbc4_ft_language [node] 
  4.35s DETAIL: load main: F.etcbc4_ft_lex [node] 
  4.55s DETAIL: load main: F.etcbc4_ft_nu [node] 
  4.72s DETAIL: load main: F.etcbc4_ft_number [n

With the following MQL query all words in the MT are collected.

In [8]:
query1 = '''
select all objects where
[book [chapter [verse
    [word language = Hebrew]
]]]
'''

In [9]:
sheaf1 = Q.mql(query1)

Booklengths are collected in book_length_dict_2, in which the booknames are converted in a shorter format. 

In [10]:
book_names = {'Genesis': 'gen', 'Exodus': 'exod', 'Leviticus': 'lev', 'Numeri': 'num', 'Deuteronomium': 'deut', 'Josua': 'josh',
              'Judices': 'judg', 'Samuel_I': '1sam', 'Samuel_II': '2sam', 'Reges_I': '1kgs', 'Reges_II': '2kgs', 'Jesaia': 'isa',
              'Jeremia': 'jer', 'Ezechiel': 'ezek', 'Hosea': 'hos', 'Joel': 'joel', 'Amos': 'amos', 'Obadia': 'oba', 'Jona': 'jona', 
              'Micha': 'mic', 'Nahum': 'nah', 'Habakuk': 'hab', 'Zephania': 'zeph', 'Haggai': 'hag', 'Sacharia': 'zech',
              'Maleachi': 'mal', 'Psalmi': 'ps', 'Iob': 'job', 'Proverbia': 'prov', 'Ruth': 'ruth', 'Canticum': 'song',
              'Ecclesiastes': 'qoh', 'Threni': 'lam', 'Esther': 'esth', 'Daniel': 'dan', 'Esra': 'ezra', 'Nehemia': 'neh',
              'Chronica_I': '1chr', 'Chronica_II': '2chr'}
book_length_dict_1 = collections.defaultdict(int)
book_length_dict_2 = {}


for ((bo, ((ch, ((ve, ((wo,),)),)),)),) in sheaf1.results():
    book_length_dict_1[F.book.v(bo)] += 1

for book in book_length_dict_1.keys():
    book_length_dict_2[book_names[book]] = book_length_dict_1[book]


In [ ]:
Counts the number of overlapping items per book

In [12]:
overlap_dict = collections.defaultdict(lambda: collections.defaultdict(list))
double_items_dict = collections.defaultdict(int)

all_overlap = ['1', '46', '53', '63', '3', '26', '11', '14', '17', '12', '22', '23', '13', '16', '45', '69', '48', '49', '51', '57', '66', '60', '70']
overlap_groups = [['1', '46', '53', '63'], ['3', '26'], ['11', '14', '17'], ['12', '22', '23'], ['13', '16'], ['45', '69'], ['48', '49'], ['51', '57', '66'], ['60', '70']]

with open('LexiconHurvitzDataset.csv', 'rt') as f:    
    reader = csv.reader(f)
    for row in reader:
        if row[3] == '0':
            if not row[4] in double_items_dict.keys():
                if not row[4] == '-':
                    double_items_dict[row[4]] = 0
            if row[1] in all_overlap:
                for feat_list in overlap_groups:
                    if row[1] in feat_list:
                        #for feature in len(feat_list):
                        overlap_dict[(row[4], row[5], row[6])][tuple(feat_list)] = [0] * len(feat_list)
                        
with open('LexiconHurvitzDataset.csv', 'rt') as f:    
    reader = csv.reader(f)
    for row in reader:
        if row[3] == '0':
            if row[1] in all_overlap:
                for feat_list in overlap_groups:
                    if row[1] in feat_list:
                        fea_list = overlap_dict[(row[4], row[5], row[6])][tuple(feat_list)]
                        index = feat_list.index(row[1])
                        fea_list[index] += int(row[7])
                        
for key in overlap_dict.keys():
    for lis in overlap_dict[key].keys():
        n_items = 0
        for item in overlap_dict[key][lis]:
            if item != 0:
                n_items += 1
        if n_items > 0:
            double = sum(overlap_dict[key][lis]) - max(overlap_dict[key][lis])
            double_items_dict[key[0]] += double  

Output: early_late_per_book.csv 
For each of the TBH/LBH books it stores the booklength,
the absolute number of early and late types and tokens and the number of early and late types and tokens per 1000 words. 

In [13]:
late_books = ['ezek', 'hag', 'zech', 'mal', 'song', 'qoh', 'esth', 'dan', 'ezra', 'neh', 'chr']
col_names = ['book', 'early_late', 'book_length', 'types', 'tokens', 'types_per_1000', 'tokens_per_1000']

csvh = open(r"early_late_per_book.csv", "w")

row= []
for name in col_names:
    row.append(name)
csvh.write('{}\n'.format(','.join(row)))

for book in late_books:
    if book == 'chr':
        row = []
        row.append(book)
        row.append('early')
        row.append(str(book_length_dict_2['1chr'] + book_length_dict_2['2chr']))
        row.append(str(len(early_freqs[book])))
        early_tokens = 0
        for key in early_freqs[book].keys():
            early_tokens += early_freqs[book][key]
        #now we substract the number of overlapping items
        early_tokens -= double_items_dict['1chr']
        early_tokens -= double_items_dict['2chr']
        row.append(str(early_tokens))
        row.append(str((len(early_freqs[book]) * 1000 / (book_length_dict_2['1chr'] + book_length_dict_2['2chr']))))
        row.append(str(-early_tokens * 1000 / (book_length_dict_2['1chr'] + book_length_dict_2['2chr'])))

        csvh.write('{}\n'.format(','.join(row)))
    else:
       row = []
       row.append(book)
       row.append('early')
       row.append(str(book_length_dict_2[book]))
       row.append(str(len(early_freqs[book])))
       early_tokens = 0
       for key in early_freqs[book].keys():
           early_tokens += early_freqs[book][key]
       early_tokens -= double_items_dict[book]
       row.append(str(early_tokens))
       row.append(str((len(early_freqs[book]) * 1000 / book_length_dict_2[book])))
       row.append(str(-early_tokens * 1000 / book_length_dict_2[book]))

       csvh.write('{}\n'.format(','.join(row))) 
for book in late_books:
    if book == 'chr':
        row = []
        row.append(book)
        row.append('late')
        row.append(str(book_length_dict_2['1chr'] + book_length_dict_2['2chr']))
        row.append(str(len(late_freqs[book])))
        late_tokens = 0
        for key in late_freqs[book].keys():
            late_tokens += late_freqs[book][key]
        row.append(str(late_tokens))
        row.append(str((len(late_freqs[book]) * 1000 / (book_length_dict_2['1chr'] + book_length_dict_2['2chr']))))
        row.append(str(late_tokens * 1000 / (book_length_dict_2['1chr'] + book_length_dict_2['2chr'])))
        
        csvh.write('{}\n'.format(','.join(row)))
    else:
       row = []
       row.append(book)
       row.append('late')
       row.append(str(book_length_dict_2[book]))
       row.append(str(len(late_freqs[book])))
       late_tokens = 0
       for key in late_freqs[book].keys():
           late_tokens += late_freqs[book][key]
       row.append(str(late_tokens))
       row.append(str((len(late_freqs[book]) * 1000 / book_length_dict_2[book])))
       row.append(str(late_tokens * 1000 / book_length_dict_2[book]))
    
       csvh.write('{}\n'.format(','.join(row))) 
csvh.close()

Figures 32-43, the diffusion plots
Input: LexiconHurvitzDataset.csv
Output: relationships_between_books_early_lat.csv

In [14]:
chron =['1chr', '2chr']
late_books = ['ezek', 'hag', 'zech', 'mal', 'song', 'qoh', 'esth', 'dan', 'ezra', 'neh', '1chr', '2chr']

#In feat_books_early and feat_books_late we collect the books in which specific variants occur. Next, in early_diff 
#and late_diff we count in how many books the features from a specific book occur
feat_books_early = collections.defaultdict(list)
feat_books_late = collections.defaultdict(list)
early_diff = collections.defaultdict(lambda:collections.Counter())
late_diff = collections.defaultdict(lambda:collections.Counter())

with open('LexiconHurvitzDataset.csv', 'rt') as f:    
    reader = csv.reader(f)
    for row in reader:
        if not row[1] == '7':
            if row[4] in late_books:
                if row[3] == '1':
                    if row[4] in chron:
                        if not 'chr' in feat_books_late[row[1]]:
                            feat_books_late[row[1]].append('chr')
                    else:
                        if not row[4] in feat_books_late[row[1]]:    
                            feat_books_late[row[1]].append(row[4])
                elif row[3] == '0':
                    if row[4] in chron:
                        if not 'chr' in feat_books_early[row[1]]:
                            feat_books_early[row[1]].append('chr')
                    else:
                        if not row[4] in feat_books_early[row[1]]:    
                            feat_books_early[row[1]].append(row[4])
                    
for key in feat_books_late.keys():
    late_diff['all_tbh_lbh'][len(feat_books_late[key])] += 1
    for book in feat_books_late[key]:
        late_diff[book][len(feat_books_late[key])] += 1
for key in feat_books_early.keys():
    early_diff['all_tbh_lbh'][len(feat_books_early[key])] += 1
    for book in feat_books_early[key]:
        early_diff[book][len(feat_books_early[key])] += 1

#The information is storedin a csv
csvh = open(r"relationships_between_books_early_lat.csv ", "w")
row = ['book', 'early_late', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
csvh.write('{}\n'.format(','.join(row)))
for book in early_diff.keys():
    row = []
    row.append(book)
    row.append('early')
    for number in range(11):
        if number + 1 in early_diff[book]:
            row.append(str(early_diff[book][number + 1]))
        else:
            row.append('0')
    csvh.write('{}\n'.format(','.join(row)))
for book in early_diff.keys():
    row = []
    row.append(book)
    row.append('late')
    for number in range(11):
        if number + 1 in late_diff[book]:
            row.append(str(late_diff[book][number + 1]))
        else:
            row.append('0')
    csvh.write('{}\n'.format(','.join(row)))    

csvh.close()

Entropy figures, Figures 45 and 46. This script uses early_freqs and late_freqs from a previous script.
Output

In [19]:
late_books = ['ezek', 'hag', 'zech', 'mal', 'song', 'qoh', 'esth', 'dan', 'ezra', 'neh', 'chr']

#In these dictionaries the entropy data are collected, per type and per token. 
entropy_dict = collections.defaultdict(lambda:collections.defaultdict(list))
entropy_dict_per_type = {}
entropy_dict_per_token = {}
for book in late_books:
    for key in list(set(list(early_freqs[book].keys()) + list(late_freqs[book].keys()))):
        if key in early_freqs[book].keys():
            entropy_dict[book][key].append(early_freqs[book][str(key)])
            
        else:
            entropy_dict[book][key].append(0)
        if key in late_freqs[book].keys():
            entropy_dict[book][key].append(late_freqs[book][str(key)])
        else:
            entropy_dict[book][key].append(0)
            
for book in late_books:
    total = 0
    for key in entropy_dict[book]:
        chance = (float(entropy_dict[book][key][1]) / (int(entropy_dict[book][key][0]) + int(entropy_dict[book][key][1])))
        if float(chance) != 0.0:
            if float(chance) != 1.0:
                total += -chance * math.log(chance,2) - (1 - chance) * math.log((1 - chance), 2)
    entropy_dict_per_type[book] = (total) / len(entropy_dict[book])

for book in late_books:
    total = 0
    total_sum_early_late = 0
    for key in entropy_dict[book]:
        chance = (float(entropy_dict[book][key][1]) / (int(entropy_dict[book][key][0]) + int(entropy_dict[book][key][1])))
        sum_early_late = entropy_dict[book][key][0] + entropy_dict[book][key][1]
        total_sum_early_late += sum_early_late
        #print(chance)
        if float(chance) != 0.0:
            if float(chance) != 1.0:
                total += (-chance * math.log(chance,2) - (1 - chance) * math.log((1 - chance), 2)) * sum_early_late
    entropy_dict_per_token[book] = total / total_sum_early_late

In [22]:
late_books = ['ezek', 'hag', 'zech', 'mal', 'song', 'qoh', 'esth', 'dan', 'ezra', 'neh', 'chr']

#similar to the previous script, but now the rate of replacement is calculated
replacement_dict = collections.defaultdict(lambda:collections.defaultdict(list))
replacement_dict_per_type = {}
replacement_dict_per_token = {}
for book in late_books:
    for key in list(set(list(early_freqs[book].keys()) + list(late_freqs[book].keys()))):
        if key in early_freqs[book].keys():
            replacement_dict[book][key].append(early_freqs[book][str(key)])
        else:
            replacement_dict[book][key].append(0)
        if key in late_freqs[book].keys():
            replacement_dict[book][key].append(late_freqs[book][str(key)])
        else:
            replacement_dict[book][key].append(0)
            
for book in late_books:
    total = 0
    for key in replacement_dict[book]:
        chance = (float(replacement_dict[book][key][1]) / (int(replacement_dict[book][key][0]) + int(replacement_dict[book][key][1])))
        if chance != 0.0:
            total += chance 
    replacement_dict_per_type[book] = (total) / len(replacement_dict[book])

for book in late_books:
    total = 0
    total_sum_early_late = 0
    for key in replacement_dict[book]:
        chance = (float(replacement_dict[book][key][1]) / (int(replacement_dict[book][key][0]) + int(replacement_dict[book][key][1])))
        sum_early_late = replacement_dict[book][key][0] + replacement_dict[book][key][1]
        total_sum_early_late += sum_early_late
        if chance != 0.0:
            total += chance * sum_early_late
    replacement_dict_per_token[book] = total / total_sum_early_late

#makes a csv to be able to compare entropy and replacement
csvh = open(r"entropy_replacement.csv", "w")
row = ['book','entropy_type','replacement_type','entropy_token','replacement_token']
csvh.write('{}\n'.format(','.join(row)))

for book in late_books:
    row = []
    row.append(book)
    row.append(str(entropy_dict_per_type[book]))
    row.append(str(replacement_dict_per_type[book]))
    row.append(str(entropy_dict_per_token[book]))
    row.append(str(replacement_dict_per_token[book]))
    csvh.write('{}\n'.format(','.join(row)))
csvh.close()

Figure 51, Frequency of all variables in all TBH/LBH books together
Input: LexiconHurvitzDataset.csv
Output: freq_all_features.csv

In [28]:
chr = ['1chr', '2chr']
late_books = ['ezek', 'hag', 'zech', 'mal', 'song', 'qoh', 'esth', 'dan', 'ezra', 'neh', 'chr', '1chr', '2chr']

general_dict = collections.defaultdict(lambda: collections.Counter())

with open('LexiconHurvitzDataset.csv', 'rt') as f:    
    reader = csv.reader(f)
    for row in reader:
        #exclude feature 7
        if not row[1] == '7':
            if row[4] in late_books:
                general_dict[row[3]][row[1]] += int(row[7])
    
csvh = open(r"freq_all_features.csv", "w")
row = ['feature', 'early_late', 'frequency']
csvh.write('{}\n'.format(','.join(row)))

for element in range(80):
    row = []
    row.append(str(element + 1))
    row.append('1')
    if not element + 1 == 7:
        if str(element + 1) in general_dict['1'].keys():
            row.append(str(general_dict['1'][str(element + 1)]))
        else:
            row.append('0')
        csvh.write('{}\n'.format(','.join(row)))
        row = []
        row.append(str(element + 1))
        row.append('0')
        if str(element + 1) in general_dict['0'].keys():
            row.append(str(general_dict['0'][str(element + 1)]))
        else:
            row.append('0')
        csvh.write('{}\n'.format(','.join(row)))    
    
csvh.close()